# OBJECTIVE:

Predicting whether lakes are polluted or not by running images of lakes through a 
convolutional neural network

# DATA GATHERING:

Data was gathered from naturalearthdata.com, Primarily derived from World Data Bank 2
http://www.naturalearthdata.com/downloads/10m-physical-vectors/10m-lakes/
It was downoaded as an SHP or shape file and then converted into an excel spreadsheet,
with each lake's coordinates. Using google maps API, the corresponding images for the coordinates were then extracted. The images were then downloaded using urllib.request library.
The Pillow library, (a fork of the PIL library which is another popular python imaging library) was also employed for further image manipulation. Then using a reverse geocode lookup, each set of coordinates were matched with their correspondng country. A Water Quality Index was then assigned for each country based on WWF (World Wide Fund For Nature) evaluation. (For furher information on Water Quality evaluation and research see below)

WQI= source http://waterriskfilter.panda.org/en/CountryProfiles#1/profileDescription: Subindex of the Environmental Performance Index (EPI). It uses three parameters measuring nutrient levels (Dissolved Oxygen, Total Nitrogen, and Total Phosphorus) and two parameters measuring water chemistry (pH and Conductivity). These parameters were selected because they cover issues of global relevance (eutrophication, nutrient pollution, acidification, and salinization) and because they are the most consistently reported. Source: Link: http://epi.yale.edu/

# DATA MUNGING:

##### The excel spreadsheet containing coordinates and countries as well as other lake related info (ie:names, city, counties
of lakes) was converted to csv along with the spreadsheet of countries and corresponding WQI (Water Quality Index). 
The dataframes were then cleaned up and merged on "country" to finally only include, coordinates, countries, names 
and WQI. WQI was then assigned either a "0" or "1", depending on the threshold. WWF defined <44 as very dangerous
water conditions and above 65 as safe. Ultimately the thresholds assigned varied between models.

# MODELING:

# Model 1: "Images Exploration ipynb" A CNN Classification Model

A threshold of WQI of 50% was implemented which resulted in a baseline of 90%.


Step 1: A dictionary was utilized to match each image with its corresponding binary WQI.
Step 2: The dictionary was then fed into a function generator that 1- found the dictionary 2- read in the images (employing cv2.imread) 3- standardized 4- reshaped and 5- assigned batches.  
Step 3: Train and Test sets were created by assigning every 5th image into a separate test folder.
Step 4: Setting up the CNN configuration. The topology was explored in order to find the best combination for the model to learn from. Factors considered were number of filters, number of kernel size and number of layers. Ultimately two models were run: The first with a simpler topology that had 24,178,901 trainable parameters. It had 2 convolutuion and pooling layers and a dense NN layer of 128. It ran 50 epochs. The second  model had 2,514 parameters (very small) since it was only reading 1,000 images total, the thought was the fewer parameters the better. It ran 30 epochs. In both models there was a dropout of .25 in the CNN and then a dropout of .5 for the NN. This was done to help prevent overfitting. The Relu activation was used throughout except for the last layer that had a "sigmoid" activation since the model was predicting a binary outcome. The model was compiled using the "Adam" optmizer, the "binary cross-entropy" loss function and the "accuracy" metric. It ran in batches of 32.

# RESULTS:

Unfortunately for both models the CNN was unable to train effectively, having an accuracy of baseline and consistently predicted "1's" as indicated by the confusion matrix. This likely was due to the fact of the large discrepency between classes. Approximately 90% of the images were of "clean" lakes which made it harder for the model to train on so few "polluted" images. In both models, the loss functions leveled out at 30 epochs.

# 2 More CNN Classification Models- Further Exploration

# Model 2: "Images_Exploration_Model_2"

# Model 2 Version 2: "Images_Exploration_Model_2_V2_less_params"

In this case a number of items were changed to see if this large discrepency between classes can be narrowed down inorder to enable the model to read the images more accurately. 
1- WQI threshold was changed from 50 to 60 effectively assigining more images to the "polluted" class.
2- All images of lakes from Canada and Russia were removed, by doing so, the decision was made to train on less images but to increase balance between the classes as now the ratio of clean to polluted was 590 to 163 (as opposed to 1191 to 163). 
3- 2 models were employed, 1 with many more parameters (176,541) and 1 with very few parameters (7,867) to see if either perfomed better with the data.

A threshold of WQI of 60% was implemented which resulted in a baseline of .78%.

The model utilized the same activation functions as previous models and was compiled using the same optimizer, loss function and metric as the earlier models.

# RESULTS:

This model yielded similar results to the past 2 models, whereby the CNN consistantly predicted only positives. Despite the fact that classes were more balanced than the last round. Perhaps there were just not enough polluted images for the model to learn from. 

# Model 3: "Images_Exploration_Model_2-Weights" A CNN classification Model employing significant weights to the minority class (polluted lakes)

A threshold of WQI of 50% was implemented which resulted in a baseline of .90%. As a result, the "0" or "polluted"
class was assigned weights of 9 times that of the "clean" class.

The model topology consisted of 3 convolutional layers and a dense layer of 32 with a dropout of .25 and .5 respectively for a total of 176,541 parameters.

In this case weights were heavily attributed to the minority class and the metrics used for evaluation were "sensitivity" and "specificity."


# RESULTS:

Interestingly enough, it seems due to the weights the results were completely reversed with the model consistantly predicting everything as "0" or "polluted." Based on the results thus far, it seems like the CNN is unable to differentiate between classes and therefor chooses the majority class and predicts solely accordingly. In this case, it learns a bit at around between 5-15 epochs and then starts consistantly predicting one class.

# Model 3: "Images_Exploration_Generator" A CNN Classification Model utilizing the ImageDataGenerator

Objective: To create more images for CNN to learn from.

The ImageDataGenerator is a generator used in CNN's that manipulates the images in a number of ways including (but not limited to) rotating, shifting, zooming, sheerness and then generates numerous additional images with these minor changes inorder to increase the number of images the model can read as well as show the different angles and perspectives so that perhaps the model can learn more. This is especially important for my dataset that has so few images for the model to train on. Please see Images_ImageDataenerator_Preview to see the various images the generator produced from a single image!

A threshold of WQI of 65% was implemented which resulted in a baseline of .83%.

The ImageDataGenerator first needs the images to be divided into train and test folders and then subdivided within each folder for polluted and clean images for a total of 4 subfolders divided between 2 folders of train and test.
It ran with 3 convolutional layers and a dense layer of 64 for a total of 152,029 parameters.
Metrics used for evaluation were "sensitivity" and "specificity."

# RESULTS:

This model yielded similar results to the past 2 models, whereby the CNN consistantly predicted only the majority class.

# CONCLUSION:

Based on evaluating models with and without weights, varied parameters, thresholds, baselines and metrics,  it seems that the next step would be to increase the database substantially and see then if the model is able to "learn" from a larger number of images. Perhaps visually, the CNN does not have enough information to detect pollution but that can only be ascertained once a substantially greater amount of images are first employed.